In [1]:
## This file implements neural networks and random forest on p002ypresabsSTCC_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 95.00% for over-sampling data and 93.5% after regularization.
## For random forest, the accuracy is 95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 96.56%.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p002ypresabsSTCC_qual.csv')
df.shape

(255, 1761)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    222
1     30
2      3
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 1760)

In [10]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,0
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,0


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1759) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 222), (1, 222), (2, 222)]


Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
############# Fully-Connected Neural Network ################

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [16]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [17]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 318us/step - loss: 1.3846 - accuracy: 0.6502 - val_loss: 0.7463 - val_accuracy: 0.6450
Epoch 2/100
466/466 [==============================] - 0s 95us/step - loss: 0.5617 - accuracy: 0.7575 - val_loss: 0.4731 - val_accuracy: 0.7950
Epoch 3/100
466/466 [==============================] - 0s 110us/step - loss: 0.4191 - accuracy: 0.8283 - val_loss: 0.4687 - val_accuracy: 0.7950
Epoch 4/100
466/466 [==============================] - 0s 101us/step - loss: 0.3843 - accuracy: 0.8498 - val_loss: 0.4351 - val_accuracy: 0.8050
Epoch 5/100
466/466 [==============================] - 0s 113us/step - loss: 0.3080 - accuracy: 0.8648 - val_loss: 0.3932 - val_accuracy: 0.8400
Epoch 6/100
466/466 [==============================] - 0s 105us/step - loss: 0.2984 - accuracy: 0.8777 - val_loss: 0.3340 - val_accuracy: 0.8850
Epoch 7/100
466/466 [==============================] - 0s 115us/step - loss: 0.2794 -

Epoch 57/100
466/466 [==============================] - 0s 90us/step - loss: 0.0407 - accuracy: 0.9893 - val_loss: 0.1760 - val_accuracy: 0.9400
Epoch 58/100
466/466 [==============================] - 0s 84us/step - loss: 0.0377 - accuracy: 0.9957 - val_loss: 0.1760 - val_accuracy: 0.9400
Epoch 59/100
466/466 [==============================] - 0s 87us/step - loss: 0.0372 - accuracy: 0.9936 - val_loss: 0.2031 - val_accuracy: 0.9400
Epoch 60/100
466/466 [==============================] - 0s 85us/step - loss: 0.0384 - accuracy: 0.9936 - val_loss: 0.1943 - val_accuracy: 0.9350
Epoch 61/100
466/466 [==============================] - 0s 84us/step - loss: 0.0405 - accuracy: 0.9957 - val_loss: 0.2061 - val_accuracy: 0.9350
Epoch 62/100
466/466 [==============================] - 0s 79us/step - loss: 0.0332 - accuracy: 0.9979 - val_loss: 0.2250 - val_accuracy: 0.9300
Epoch 63/100
466/466 [==============================] - 0s 81us/step - loss: 0.0413 - accuracy: 0.9936 - val_loss: 0.1593 - val_ac

In [19]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

200/200 [==============================] - 0s 60us/step
over-sampling test accuracy: 95.00%


In [20]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [21]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 398us/step - loss: 6.3753 - accuracy: 0.4721 - val_loss: 3.5044 - val_accuracy: 0.6150
Epoch 2/100
466/466 [==============================] - 0s 114us/step - loss: 4.4446 - accuracy: 0.5858 - val_loss: 2.0398 - val_accuracy: 0.7450
Epoch 3/100
466/466 [==============================] - 0s 100us/step - loss: 3.3138 - accuracy: 0.6652 - val_loss: 1.5731 - val_accuracy: 0.7300
Epoch 4/100
466/466 [==============================] - 0s 102us/step - loss: 3.1581 - accuracy: 0.6695 - val_loss: 1.3783 - val_accuracy: 0.7850
Epoch 5/100
466/466 [==============================] - 0s 113us/step - loss: 3.1832 - accuracy: 0.6996 - val_loss: 1.3615 - val_accuracy: 0.7950
Epoch 6/100
466/466 [==============================] - 0s 110us/step - loss: 3.2319 - accuracy: 0.6674 - val_loss: 1.3856 - val_accuracy: 0.8050
Epoch 7/100
466/466 [==============================] - 0s 100us/step - loss: 3.5248 

Epoch 57/100
466/466 [==============================] - 0s 206us/step - loss: 1.6223 - accuracy: 0.7854 - val_loss: 0.9994 - val_accuracy: 0.8900
Epoch 58/100
466/466 [==============================] - 0s 91us/step - loss: 1.5870 - accuracy: 0.7532 - val_loss: 1.1433 - val_accuracy: 0.8850
Epoch 59/100
466/466 [==============================] - 0s 85us/step - loss: 1.6189 - accuracy: 0.7403 - val_loss: 1.0298 - val_accuracy: 0.9050
Epoch 60/100
466/466 [==============================] - 0s 83us/step - loss: 1.4059 - accuracy: 0.7940 - val_loss: 1.0490 - val_accuracy: 0.8850
Epoch 61/100
466/466 [==============================] - 0s 96us/step - loss: 1.2545 - accuracy: 0.8090 - val_loss: 0.9817 - val_accuracy: 0.9050
Epoch 62/100
466/466 [==============================] - 0s 97us/step - loss: 1.3956 - accuracy: 0.7597 - val_loss: 1.2825 - val_accuracy: 0.8750
Epoch 63/100
466/466 [==============================] - 0s 118us/step - loss: 1.4060 - accuracy: 0.7661 - val_loss: 1.1437 - val_

In [23]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

200/200 [==============================] - 0s 46us/step
over-sampling test accuracy: 94.00%


In [24]:
############## Random Forest ##############

In [25]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.00%


In [28]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.9787234  1.         0.95698925 0.94623656 0.93548387]
0.9634866163349347
